In [1]:
pip install ucimlrepo

In [2]:
!sudo apt update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [830 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,798 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,374 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa

### Load data

In [3]:
from ucimlrepo import fetch_ucirepo

data = fetch_ucirepo(id=848)
pdf = data.data.features
pdf = pdf.drop(columns = [
    'cap-diameter',
    'stem-height',
    'stem-width',
])
pdf = pdf.astype('<U22')


### bank dataset #######
#data = fetch_ucirepo(id=222)

#pdf = data.data.features
#
#def bin_age(age):
#  if age < 18:
#      return "Under 18"
#  elif age >= 18 and age < 30:
#      return "18-29"
#  elif age >= 30 and age < 40:
#      return "30-39"
#  elif age >= 40 and age < 50:
#      return "40-49"
#  elif age >= 50 and age < 60:
#      return "50-59"
#  else:
#      return "60+"
#
#def bin_duration(duration):
#  if duration < 103:
#      return "low"
#  elif duration >= 103:
#      return "mid"
#  else:
#      return "high"
#
#def bin_campaign(campaign):
#  if campaign <= 3:
#      return "normal"
#  else:
#      return "high"
#
#def prep_pdays(pdays):
#  if pdays == -1:
#    return "no"
#  else:
#    return "yes"
#
#def bin_balance(balance):
#  if balance <= 72:
#    return "low"
#  elif balance <= 448:
#    return "mid"
#  elif balance <= 1428:
#    return "high"
#  else:
#    return "very high"
#
#def bin_previous(previous):
#  if previous > 0:
#    return "yes"
#  else:
#    return "no"
#
#
#pdf['age'] = pdf['age'].apply(lambda x: bin_age(x))
#pdf['balance'] = pdf['balance'].apply(lambda x: bin_balance(x))
#pdf['duration'] = pdf['duration'].apply(lambda x: bin_duration(x))
#pdf['campaign'] = pdf['campaign'].apply(lambda x: bin_campaign(x))
#pdf['pdays'] = pdf['pdays'].apply(lambda x: x == -1)
#pdf['previous'] = pdf['previous'].apply(lambda x: bin_previous(x))
#
#pdf = pdf.drop(columns = ['day_of_week', 'poutcome'])
############################

####################

####################


#from google.colab import drive
#drive.mount('/content/drive')
#
#import pandas as pd
#
#pdf = pd.read_csv('/content/drive/MyDrive/BigData/K-Mode/train.csv')
#pdf = pdf.drop(columns=['id', 'target'])
#pdf = pdf.astype('<U22')

In [4]:
pdf

,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,x,g,o,f,e,nan,w,s,y,w,u,w,t,g,nan,d,w
1,x,g,o,f,e,nan,w,s,y,w,u,w,t,g,nan,d,u
2,x,g,o,f,e,nan,w,s,y,w,u,w,t,g,nan,d,w
3,f,h,e,f,e,nan,w,s,y,w,u,w,t,p,nan,d,w
4,x,h,o,f,e,nan,w,s,y,w,u,w,t,p,nan,d,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,s,s,y,f,f,f,f,nan,nan,y,nan,nan,f,f,nan,d,a
61065,f,s,y,f,f,f,f,nan,nan,y,nan,nan,f,f,nan,d,a
61066,s,s,y,f,f,f,f,nan,nan,y,nan,nan,f,f,nan,d,u
61067,f,s,y,f,f,f,f,nan,nan,y,nan,nan,f,f,nan,d,u


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("K-Mode-global") \
    .getOrCreate()

sc = spark.sparkContext

In [6]:
df = spark.createDataFrame(pdf)
P = len(df.columns)
N_ROWS = len(pdf)
df.cache()
rdd = df.rdd.repartition(10)
rdd.cache()

MapPartitionsRDD[18] at coalesce at NativeMethodAccessorImpl.java:0

In [8]:
import numpy as np

In [9]:
def to_numpy(row):
  a = list(row.asDict().values())
  return np.array(a, dtype="<U22")

### Initiate centroid by randomly select datapoint

In [11]:
from math import e
import random
import numpy as np

K = 3 #number of cluster

for (idx, row) in enumerate(rdd.takeSample(withReplacement=False, num=K, seed=42)):
  if idx == 0:
    centroid = np.array([to_numpy(row)]) #shape (1, P)
  else:
    c_ = to_numpy(row)
    c_ = np.array([c_]) #shape (1, P)
    centroid = np.concatenate([centroid, c_], axis=0)

init_centroid = centroid.copy() ##for report

centroid = sc.broadcast(centroid) #shape (K, P)
centroid.value

array([['f', 'd', 'n', 't', 'd', 'c', 'p', 'nan', 'nan', 'g', 'nan',
        'nan', 'f', 'f', 'nan', 'd', 'u'],
       ['x', 'e', 'e', 'f', 'a', 'nan', 'y', 'nan', 'nan', 'e', 'nan',
        'w', 't', 'z', 'nan', 'd', 'a'],
       ['x', 'nan', 'g', 't', 'p', 'nan', 'g', 'nan', 'y', 'w', 'nan',
        'nan', 'f', 'f', 'nan', 'd', 'a']], dtype='<U22')

In [12]:
rdd = rdd.map(lambda row: to_numpy(row))

In [13]:
def hamming_distance(x1, x2):
  return np.count_nonzero(x1!=x2)

def get_closest_cluster_and_count(x, centroid):
  min_hamming_distance = np.inf
  closest_cluster = 0
  for i, mode in enumerate(centroid):
     distance = hamming_distance(x, mode)
     if distance < min_hamming_distance:
        min_hamming_distance = distance
        closest_cluster = i

  P = len(x)

  count_elem_hash = {}
  for i in range(P):
    count_elem_hash[i] = {x[i] : 1}

  return (closest_cluster, count_elem_hash)

def merge_count_elem_hash(count_elem_hash_A, count_elem_hash_B):
  for idx in count_elem_hash_A.keys():
    for key in count_elem_hash_B[idx].keys():
      if key in count_elem_hash_A[idx]:
        count_elem_hash_A[idx][key] += count_elem_hash_B[idx][key]
      else:
        count_elem_hash_A[idx][key] = count_elem_hash_B[idx][key]
  return count_elem_hash_A

def get_centroid(count_elem_hash):
  P = len(count_elem_hash)
  centroid = np.full((P,), "", dtype="<U22")
  for idx, count_hash in count_elem_hash.items():
    mode = get_mode(count_hash)
    centroid[idx] = mode
  return centroid

def get_mode(count_hash):
  mode = ""
  highest_count = 0
  for value, count in count_hash.items():
    if count > highest_count:
      mode = value
      highest_count = count
  return mode

def get_loss(x, centroid):
  min_hamming_distance = np.inf
  for i, mode in enumerate(centroid):
     distance = hamming_distance(x, mode)
     if distance < min_hamming_distance:
        min_hamming_distance = distance

  return min_hamming_distance

### KMode global approach

In [14]:
stop_distance = P//10

N_iters = 10

losses = []

for iter in range(N_iters):

  ## for report
  loss = rdd.map(
      lambda x: get_loss(x, centroid.value)
      ).reduce(lambda x, y: x+y)/N_ROWS
  losses.append(loss)
  ## for report

  clustered_and_hash_count_rdd = rdd.map(lambda x: get_closest_cluster_and_count(x, centroid.value)) #-> (k, v) = (cluster_i, count_hash)
  counted_elem_rdd = clustered_and_hash_count_rdd.reduceByKey(lambda x, y: merge_count_elem_hash(x, y))
  cluster_and_new_centroid_rdd =  counted_elem_rdd.map(lambda x: (x[0], get_centroid(x[1])))
  centroid_hash_form = cluster_and_new_centroid_rdd.collect()

  for idx in range(K):
    if idx == 0:
      new_centroid = np.array([centroid_hash_form[idx][1]])
    else:
      mode = np.array([centroid_hash_form[idx][1]])
      new_centroid = np.concatenate([new_centroid, mode])

  old_centroid = centroid.value.copy()
  centroid = sc.broadcast(new_centroid)

  distance = hamming_distance(old_centroid, new_centroid)

  print('iteration : ', {iter+1}, " hamming distance between new and previous centroid:  ", distance)

  if distance <= stop_distance:
    break

iteration :  {1}  hamming distance between new and previous centroid:   14
iteration :  {2}  hamming distance between new and previous centroid:   2
iteration :  {3}  hamming distance between new and previous centroid:   0


In [ ]:
losses

In [ ]:
centroid_hash_form

### KMode sequential version (We use the same intital centroid as global)

In [42]:
import random
import numpy as np
from collections import Counter

class KMode:

    def __init__(self, K, centroid=None) -> None:
        self.K = K
        self.centroid = centroid

    def init_centroid(self, X):
        col_len = X.shape[1]
        centroid = np.array([], dtype="<U22")
        for i in range(self.K):
            mode = []
            for col in range(col_len):
                rand_val = random.choice(np.unique(X[:, col]))
                mode.append(rand_val)
            if i == 0:
                centroid = np.array(mode, dtype="<U22")
                del mode
            else:
                mode = np.array(mode, dtype="<U22")
                centroid = np.vstack((centroid, mode))
        self.centroid = centroid
        return centroid

    def get_closest_centroid(self, x):
        min_hamming_distance = np.inf
        closest_cluster = 0
        for i, mode in enumerate(self.centroid):
            distance = self.hamming_distance(x, mode)
            if distance < min_hamming_distance:
                min_hamming_distance = distance
                closest_cluster = i
        return closest_cluster

    def fit(self, X, n_iters=100, stopping_criterion=1) -> None:
        losses = [] #for report
        if self.centroid is None:
          self.init_centroid(X)
        self.clustered = np.zeros(X.shape[0])
        for iter in range(n_iters):
            loss = self.fit_one_step(X)
            print(f"iter: {iter} loss: {loss}")

            if loss < stopping_criterion:
                break

    def fit_one_step(self, X) -> int:
        self.prev_centroid = self.centroid.copy()
        for i, x in enumerate(X):
            closest_centroid = self.get_closest_centroid(x)
            self.clustered[i] = closest_centroid
        for i in range(self.K):
            idx = np.where(self.clustered == i)[0]
            if idx.shape[0] > 0:
                new_mode = self.get_mode_from_arr(X[idx])
                self.centroid[i] = new_mode
        return self.hamming_distance(self.centroid, self.prev_centroid)

    def transform(self, X_test):
        clustered = np.zeros(X_test.shape[0])
        for i, x in enumerate(X_test):
            closest_centroid = self.get_closest_centroid(x)
            clustered[i] = closest_centroid
        return clustered.astype('int')

    @staticmethod
    def hamming_distance(a, b) -> int:
        return np.count_nonzero(a != b)

    @staticmethod
    def get_mode_from_arr(arr):
        def get_mode_from_vec(vec):
            counted = Counter(vec)
            return counted.most_common(1)[0][0]
        P = arr.shape[1]
        mode_arr = np.full((arr.shape[1], ), "", dtype="<U22")
        for p in range(P):
          mode = get_mode_from_vec(arr[:, p])
          mode_arr[p] = mode
        return mode_arr



In [ ]:
kmode = KMode(K=K, centroid=init_centroid.copy())
X = pdf.values
kmode.fit(X)

Centroid from sequential

In [ ]:
kmode.centroid

#### Centroid from global approach

In [ ]:
centroid_hash_form